In [17]:
import OpenDVCW
import tensorflow as tf
from tensorflow.keras.models import Model

In [2]:
model_dir = "/mnt/WindowsDev/PreTainedModels/OpenDVCW/HAAR200k/model_save_checkpoints_wavelets_haar_Lmbd_256_nfilt_256_epcs_5_stps_200000_I_QP_42_240x240_CosineDecay_20221207-161406/"
model = tf.keras.models.load_model(model_dir)

2022-12-21 18:56:05.728893: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-21 18:56:05.732264: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-21 18:56:05.732380: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-21 18:56:05.732699: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [33]:

i_frame = "/mnt/WindowsDev/DataSets/Beauty_1920x1080_120fps_420_8bit_YUV_RAW/im0.png"
p_frame = "/mnt/WindowsDev/DataSets/Beauty_1920x1080_120fps_420_8bit_YUV_RAW/im15.png"
comp_bin = "/workspaces/OpenDVCW/Test_com/tmp/tmp.bin"
recon_frame = "/workspaces/OpenDVCW/Test_com/tmp/recon.png"
motion_flow = "/workspaces/OpenDVCW/Test_com/tmp/flow.png"
width = 240
height = 240
channels = 3

Y0_com = OpenDVCW.read_png_crop(i_frame, width, height)
Y1_raw = OpenDVCW.read_png_crop(p_frame, width, height) 
Y0_com = tf.expand_dims(Y0_com, 0)
Y1_raw = tf.expand_dims(Y1_raw, 0)
Y0_com = tf.cast(Y0_com / 255, dtype=tf.float32)
Y1_raw = tf.cast(Y1_raw / 255, dtype=tf.float32)

In [35]:
# OpenDVCW.compress(model, i_frame, p_frame, comp_bin, width, height)
# OpenDVCW.decompress(model, i_frame, comp_bin, recon_frame, width, height)
flow_tensor = model.optical_flow([Y0_com, Y1_raw])


In [5]:
flow_tensor_c = flow_tensor[0, :240, :240, :] * 255
flow_tensor_u8 = tf.saturate_cast(tf.round(flow_tensor_c), tf.uint8)
OpenDVCW.write_png(motion_flow, flow_tensor_u8)

In [6]:
print(tf.reduce_max(flow_tensor_c))

tf.Tensor(0.0, shape=(), dtype=float32)
